# 第2章: UNIXコマンド
popular-names.txtは，アメリカで生まれた赤ちゃんの「名前」「性別」「人数」「年」をタブ区切り形式で格納したファイルである．以下の処理を行うプログラムを作成し，popular-names.txtを入力ファイルとして実行せよ．さらに，同様の処理をUNIXコマンドでも実行し，プログラムの実行結果を確認せよ．

In [0]:
!wget https://nlp100.github.io/data/popular-names.txt

--2020-05-28 00:02:13--  https://nlp100.github.io/data/popular-names.txt
Resolving nlp100.github.io (nlp100.github.io)... 185.199.110.153, 185.199.109.153, 185.199.108.153, ...
Connecting to nlp100.github.io (nlp100.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55026 (54K) [text/plain]
Saving to: ‘popular-names.txt.1’

popular-names.txt.1 100%[===================>]  53.74K  --.-KB/s    in 0.02s   

2020-05-28 00:02:14 (3.49 MB/s) - ‘popular-names.txt.1’ saved [55026/55026]



## 10. 行数のカウント
***
行数をカウントせよ．確認にはwcコマンドを用いよ．

In [0]:
import pandas as pd

df = pd.read_table('./popular-names.txt', header=None, sep='\t', names=['name', 'sex', 'number', 'year'])
print(len(df))

2780


In [0]:
# 確認
!cat ./popular-names.txt | wc -l

2780


## 11. タブをスペースに置換
***
タブ1文字につきスペース1文字に置換せよ．確認にはsedコマンド，trコマンド，もしくはexpandコマンドを用いよ．

In [0]:
# 確認
!sed -e 's/\t/ /g' ./popular-names.txt | head -n 5

Mary F 7065 1880
Anna F 2604 1880
Emma F 2003 1880
Elizabeth F 1939 1880
Minnie F 1746 1880


## 12. 1列目をcol1.txtに，2列目をcol2.txtに保存
***
各行の1列目だけを抜き出したものをcol1.txtに，2列目だけを抜き出したものをcol2.txtとしてファイルに保存せよ．確認にはcutコマンドを用いよ．

In [0]:
col1 = df['name'].copy()
col1.to_csv('./col1.txt', index=False)
col2 = df['sex'].copy()
col2.to_csv('./col2.txt', index=False)

In [0]:
# 確認
!cut -f 1 ./popular-names.txt > ./col1_chk.txt
!cat ./col1_chk.txt | head -n 5

Mary
Anna
Emma
Elizabeth
Minnie


In [0]:
!cut -f 2 ./popular-names.txt > ./col2_chk.txt
!cat ./col2_chk.txt | head -n 5

F
F
F
F
F


## 13. col1.txtとcol2.txtをマージ
***
12で作ったcol1.txtとcol2.txtを結合し，元のファイルの1列目と2列目をタブ区切りで並べたテキストファイルを作成せよ．確認にはpasteコマンドを用いよ．

In [0]:
col1 = pd.read_table('./col1.txt')
col2 = pd.read_table('./col2.txt')
merged_1_2 = pd.concat([col1, col2], axis=1)
merged_1_2.to_csv('./merged_1_2.txt', sep='\t', index=False)
print(merged_1_2.head())

        name sex
0       Mary   F
1       Anna   F
2       Emma   F
3  Elizabeth   F
4     Minnie   F


In [0]:
# 確認
!paste ./col1_chk.txt ./col2_chk.txt | head -n 5

Mary	F
Anna	F
Emma	F
Elizabeth	F
Minnie	F


## 14. 先頭からN行を出力
***
自然数Nをコマンドライン引数などの手段で受け取り，入力のうち先頭のN行だけを表示せよ．確認にはheadコマンドを用いよ．

In [0]:
def output_head(N):
  print(df.head(N))
output_head(5)

        name sex  number  year
0       Mary   F    7065  1880
1       Anna   F    2604  1880
2       Emma   F    2003  1880
3  Elizabeth   F    1939  1880
4     Minnie   F    1746  1880


In [0]:
# 確認
!head -n 5 ./popular-names.txt

Mary	F	7065	1880
Anna	F	2604	1880
Emma	F	2003	1880
Elizabeth	F	1939	1880
Minnie	F	1746	1880


## 15. 末尾のN行を出力
***
自然数Nをコマンドライン引数などの手段で受け取り，入力のうち末尾のN行だけを表示せよ．確認にはtailコマンドを用いよ．

In [0]:
def output_tail(N):
  print(df.tail(N))
output_tail(5)

          name sex  number  year
2775  Benjamin   M   13381  2018
2776    Elijah   M   12886  2018
2777     Lucas   M   12585  2018
2778     Mason   M   12435  2018
2779     Logan   M   12352  2018


In [0]:
# 確認
!tail -n 5 ./popular-names.txt

Benjamin	M	13381	2018
Elijah	M	12886	2018
Lucas	M	12585	2018
Mason	M	12435	2018
Logan	M	12352	2018


## 16. ファイルをN分割する
***
自然数Nをコマンドライン引数などの手段で受け取り，入力のファイルを行単位でN分割せよ．同様の処理をsplitコマンドで実現せよ．

In [0]:
def split_file(N):
  tmp = df.reset_index(drop=False)
  df_cut = pd.qcut(tmp.index, N, labels=[i for i in range(N)])
  print(df_cut.value_counts())
  df_cut = pd.concat([df, pd.Series(df_cut, name='sp')], axis=1)
  return df_cut
df_cut = split_file(10)
print(df_cut.head())

0    278
1    278
2    278
3    278
4    278
5    278
6    278
7    278
8    278
9    278
dtype: int64
        name sex  number  year sp
0       Mary   F    7065  1880  0
1       Anna   F    2604  1880  0
2       Emma   F    2003  1880  0
3  Elizabeth   F    1939  1880  0
4     Minnie   F    1746  1880  0


In [0]:
!split -l 200 -d ./popular-names.txt sp

## 17. １列目の文字列の異なり
***
1列目の文字列の種類（異なる文字列の集合）を求めよ．確認にはcut, sort, uniqコマンドを用いよ．

In [0]:
print(len(df.drop_duplicates(subset='name')))

136


In [0]:
# 確認
!cut -f 1 ./popular-names.txt | sort | uniq | wc -l

136


## 18. 各行を3コラム目の数値の降順にソート
***
各行を3コラム目の数値の逆順で整列せよ（注意: 各行の内容は変更せずに並び替えよ）．確認にはsortコマンドを用いよ（この問題はコマンドで実行した時の結果と合わなくてもよい）．

In [0]:
df.sort_values(by='number', ascending=False, inplace=True)
print(df.head())

         name sex  number  year
1340    Linda   F   99689  1947
1360    Linda   F   96211  1948
1350    James   M   94757  1947
1550  Michael   M   92704  1957
1351   Robert   M   91640  1947


In [0]:
# 確認
!cat ./popular-names.txt | sort -rnk 3 | head -n 5

Linda	F	99689	1947
Linda	F	96211	1948
James	M	94757	1947
Michael	M	92704	1957
Robert	M	91640	1947


## 19. 各行の1コラム目の文字列の出現頻度を求め，出現頻度の高い順に並べる
***
各行の1列目の文字列の出現頻度を求め，その高い順に並べて表示せよ．確認にはcut, uniq, sortコマンドを用いよ．

In [0]:
print(df['name'].value_counts())

James      118
William    111
John       108
Robert     108
Mary        92
          ... 
Tracy        1
Julie        1
Carolyn      1
Lori         1
Lucas        1
Name: name, Length: 136, dtype: int64


In [0]:
# 確認
!cut -f 1 ./popular-names.txt | sort | uniq -c | sort -rn | head -n 5

    118 James
    111 William
    108 Robert
    108 John
     92 Mary
